In [ ]:
import time
import os
import csv
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from keras.utils import np_utils
from PIL import Image
from keras import backend as K
from keras.preprocessing.image import array_to_img
from IPython.display import Image

In [ ]:
#作成するモデルの保存用
#フェーズ視認用
result_folder="2DCNN_results"
date_folder=20221226
title="2DCNN_Ct_machinelearning" 
models="ResNet50"
case=2  #分類器の選択
#optimizer の選択
opt="Adam"
#opt='AMSGrad'
#opt="RMS"
#opt='SGD'
#opt='adamax'

dataset="allequalver"

#学習に関するハイパパラメータ
epochs=300

#画像サイズ
size=224
num_class=2

In [ ]:
# CSVファイル読み込みの関数
# ラベル付け
def label_value(serch):
    if "acceptable" in serch:
        value=0
    else :
        value=1
        
    return value

#データの読み込みのために、csv読み込み
def read_path_and_tag(csv_f):
    train=[]
    train_label=[]
    train_tag=[]
    valid=[]
    valid_label=[]
    valid_tag=[]
    test=[]
    test_label=[]
    test_tag=[]

    with open(csv_f) as f:
        reader =csv.reader(f)
        for row in reader:

            if row[1] == "test":
                test.append(row[0])
                test_label.append(label_value(row[0]))
                test_tag.append(row[1])

            elif row[1]=="valid":
                valid.append(row[0])
                valid_label.append(label_value(row[0]))
                valid_tag.append(row[1])

            elif row[1]=="train":
                train.append(row[0])
                train_label.append(label_value(row[0]))
                train_tag.append(row[1])

    return train,train_label,train_tag,valid,valid_label,valid_tag,test,test_label,test_tag

In [ ]:
# CSVファイルからパスの取得
CTace_path =  read_path_and_tag("file_way_CT_1of2_2DForUse_retable.csv")
CTcol_path =  read_path_and_tag("file_way_CT_2of2_2DForUse_retable.csv")

array_name = ["train","train_label","train_tag","valid","valid_label","valid_tag","test","test_label","test_tag"]
array = ["train","train_label","train_tag","valid","valid_label","valid_tag","test","test_label","test_tag"]

for i in range(9):
    array[i] = CTace_path[i]+CTcol_path[i]
    
train = array[0]
train_label = array[1]
train_tag = array[2]
valid = array[3]
valid_label = array[4]
valid_tag = array[5]
test = array[6]
test_label = array[7]
test_tag = array[8]

for i in range(9):
    print(array_name[i],len(array[i]))

In [ ]:
# Add validation data to training data for cross_validation
trainvalid = []
trainvalid_label = []
trainvalid_tag = []

trainvalid = train + valid
trainvalid_label = train_label + valid_label
trainvalid_tag = train_tag + valid_tag
# print(len(trainvalid),trainvalid)
# print(len(trainvalid_label),trainvalid_label)

In [ ]:
# 少ないほうを多いほうと同数にコピー
trainvalid_0=[]
trainvalid_label_0=[]
trainvalid_tag_0=[]
trainvalid_1=[]
trainvalid_label_1=[]
trainvalid_tag_1=[]

cnt=0
for i in trainvalid_label:
    if i == 0:
        tmp=trainvalid[cnt]
        trainvalid_0.append(tmp)
        labeltmp=trainvalid_label[cnt]
        trainvalid_label_0.append(labeltmp)
        tagtmp=trainvalid_tag[cnt]
        trainvalid_tag_0.append(tagtmp)
    elif i == 1:
        tmp=trainvalid[cnt]
        trainvalid_1.append(tmp)
        labeltmp=trainvalid_label[cnt]
        trainvalid_label_1.append(labeltmp)
        tagtmp=trainvalid_tag[cnt]
        trainvalid_tag_1.append(tagtmp)
    cnt+=1
    
trainvalid_imfalted=[]
trainvalid_label_imfalted=[]
trainvalid_tag_imfalted=[]
if len(trainvalid_0)<len(trainvalid_1):
    trainvalid_add=len(trainvalid_1)-len(trainvalid_0)
    for i in range(100):
        for n in range(len(trainvalid_0)):
            if len(trainvalid_imfalted) >= trainvalid_add:
                break        
            trainvalid_imfalted.append(trainvalid_0[n])
            trainvalid_label_imfalted.append(trainvalid_label_0[n])
            trainvalid_tag_imfalted.append(trainvalid_tag_0[n])

    #もとのデータと結合＝＞同数の調整完了
    trainvalid = trainvalid + trainvalid_imfalted  # リスト型同士の足し合わせ

    trainvalid_label = trainvalid_label + trainvalid_label_imfalted  # 初期のリスト型

    trainvalid_tag = trainvalid_tag + trainvalid_tag_imfalted

    print(len(trainvalid),len(trainvalid_label),len(trainvalid_tag))

elif len(trainvalid_1)<len(trainvalid_0):
    trainvalid_add=len(trainvalid_0)-len(trainvalid_1)
    for i in range(100):
        for n in range(len(trainvalid_1)):
            if len(trainvalid_imfalted) >= trainvalid_add:
                break        
            trainvalid_imfalted.append(trainvalid_1[n])
            trainvalid_label_imfalted.append(trainvalid_label_1[n])
            trainvalid_tag_imfalted.append(trainvalid_tag_1[n])
            
    #もとのデータと結合＝＞同数の調整完了
    trainvalid = trainvalid + trainvalid_imfalted  # リスト型同士の足し合わせ

    trainvalid_label = trainvalid_label + trainvalid_label_imfalted  # 初期のリスト型

    trainvalid_tag = trainvalid_tag + trainvalid_tag_imfalted

    print(len(trainvalid),len(trainvalid_label),len(trainvalid_tag))

In [ ]:
# チェック
trainvalid_0=[]
trainvalid_1=[]
trainvalid_label_0=[]
trainvalid_label_1=[]


for i in range(len(trainvalid_label)):
    if trainvalid_label[i]==0:
        trainvalid_0.append(trainvalid[i])
        trainvalid_label_0.append(0)
    elif trainvalid_label[i]==1:
        trainvalid_1.append(trainvalid[i])
        trainvalid_label_1.append(1)
print(len(trainvalid_0),len(trainvalid_1),len(trainvalid_label_0),len(trainvalid_label_1))
#print(trainvalid_0)

In [ ]:
# trainデータの読み込み
from matplotlib import pyplot as plt
trainvalid_0_img = []
trainvalid_1_img = []
for img in trainvalid_0:
    trainvalid_0_img.append(cv2.imread(img))

for img in trainvalid_1:
    trainvalid_1_img.append(cv2.imread(img))
    
print(len(trainvalid_0),len(trainvalid_1))

In [ ]:
#  パスのダブりを解消、区別をつける
for i in range(len(trainvalid_0)):
    cnt = 1
    for t in range(len(trainvalid_0)):
        if i!=t:
            if trainvalid_0[i]==trainvalid_0[t]:
                if ".png" in trainvalid_0[t]:
                    if cnt==1:
                        trainvalid_0[t]=trainvalid_0[t].replace('.png','_copy'+str(cnt)+'.png')
                        cnt += 1
                        
                    elif cnt==2:
                        trainvalid_0[t]=trainvalid_0[t].replace('.png','_copy'+str(cnt)+'.png')
                        cnt += 1
                    
                    elif cnt==3:
                        trainvalid_0[t]=trainvalid_0[t].replace('.png','_copy'+str(cnt)+'.png')

# 解消できたかチェック
for i in range(len(trainvalid_0)):
    for t in range(len(trainvalid_0)):
        if i!=t:
            if trainvalid_0[i]==trainvalid_0[t]:
                print(t,trainvalid_0[t])


In [ ]:
#  パスのダブりを解消、区別をつける
for i in range(len(trainvalid_1)):
    cnt = 1
    for t in range(len(trainvalid_1)):
        if i!=t:
            if trainvalid_1[i]==trainvalid_1[t]:
                if ".png" in trainvalid_1[t]:
                    if cnt==1:
                        trainvalid_1[t]=trainvalid_1[t].replace('.png','_copy'+str(cnt)+'.png')
                        cnt += 1
                        
                    elif cnt==2:
                        trainvalid_1[t]=trainvalid_1[t].replace('.png','_copy'+str(cnt)+'.png')
                        cnt += 1
                    
                    elif cnt==3:
                        trainvalid_1[t]=trainvalid_1[t].replace('.png','_copy'+str(cnt)+'.png')

# 解消できたかチェック
for i in range(len(trainvalid_1)):
    for t in range(len(trainvalid_1)):
        if i!=t:
            if trainvalid_1[i]==trainvalid_1[t]:
                print(t,trainvalid_1[t])


In [ ]:
#testも同数に
test_0=[]
test_1=[]
pickup_test=[]
test_equal=[]
random.seed(1)

for i in test:
    if "acceptable" in i:
        test_0.append(i)
    else:
        test_1.append(i)

if len(test_0)<len(test_1):
    pickup_test=random.sample(test_1,len(test_0))
    test_equal=test_0+pickup_test      
elif len(test_1)<len(test_0):
    pickup_test=random.sample(test_0,len(test_1))
    test_equal=test_1+pickup_test 

# test_equalのラベル作成
test_equal_label=[]
for i in test_equal:
    if "acceptable" in i:
        test_equal_label.append(0)
            
    else:
        test_equal_label.append(1)
            
test=[]
test_label=[]
for i in range(len(test_equal_label)):
    test.append(test_equal[i])
    test_label.append(valid_equal_label[i])
print(len(test),len(test_label))

In [ ]:
#testデータの読み込み
test_img = []
for img in test_equal:
    test_img.append(cv2.imread(img))
    
print(len(test_img))

In [ ]:
# 使用する画像をresizeする
for i in range(len(trainvalid_1_img)):
    trainvalid_1_img[i] = cv2.resize(trainvalid_1_img[i],(224,224))
    trainvalid_0_img[i] = cv2.resize(trainvalid_0_img[i],(224,224))
    
for i in range(len(test_img)):
    test_img[i] = cv2.resize(test_img[i],(224,224))

In [ ]:
print(type(trainvalid_0_img))
print(trainvalid_0_img[0].shape)
print(type(test_img[0]))
print(test_img[0].shape)

In [ ]:
#画像の表示　内容の確認
from matplotlib import pyplot as plt
for i in range(4):
    plt.subplot(2,4,i+1)
    plt.imshow(trainvalid_0_img[i])

In [ ]:
#np配列に変換
trainvalid_0_img2 = np.asarray(trainvalid_0_img)
trainvalid_1_img2 = np.asarray(trainvalid_1_img)
test_img = np.asarray(test_img)
trainvalid_label_0_ndarray = np.asarray(trainvalid_label_0)
trainvalid_label_1_ndarray = np.asarray(trainvalid_label_1)
print(type(trainvalid_0_img2))
print(type(trainvalid_label_0_ndarray))
print(trainvalid_label_0_ndarray)

In [ ]:
# 保存に使うパスの作成用
def folder_path(result_folder,date_folder,title,model,case,opt,dataset,train_amount,cnt,rate,batch,epochs):#  保存に使う
    result = result_folder + '/'
    date = str(date_folder) +'_results/'
    path = title + "-NN" + str(case) + "/" + opt + '/'
    select =  dataset + '/'+"train_" + str(train_amount) + '_rate_' + str(rate) + '_batch_' + str(batch) + '_epochs_' + str(epochs) + '_optimizer_' + opt +"/crosval_cnt"+str(cnt)+"/"
    
    save_name = result + date + path + select
    save_view_name = "NN" + str(case) + "_" + opt + '_' + select
    
    return save_name,save_view_name

In [ ]:
# モデルの構築
#モデルの構築
from keras.applications.resnet50 import ResNet50
from keras.models import Model,Sequential
import tensorflow as tf
import keras
from keras.layers import Conv2D,MaxPooling2D,Input,Dense,Dropout,Flatten,Activation,GlobalAveragePooling2D
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers
from keras.layers import BatchNormalization

def make_model(size,numclass,optimizer,case,mode,read_name):
    #ResNet50のロード　（FC層を除く
    input_tensor=Input(shape=(size,size,3))
    resnet50=ResNet50(include_top=False,
                      weights='imagenet',
                      input_tensor=input_tensor)
    
     #最下層の出力層のオリジナルモデル部分をsequentialモデルで作成
    top_model=Sequential()
    top_model.add(Flatten(input_shape=resnet50.output_shape[1:]))
    if case==1:
        top_model.add(Dense(numclass,activation='softmax'))
    elif case==2:
        top_model.add(Dense(256,activation='relu'))
        top_model.add(Dropout(0.1))
        top_model.add(Dense(128,activation='relu'))
        top_model.add(Dropout(0.1))
        top_model.add(Dense(64,activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(numclass,activation='softmax'))
    elif case==3:
        top_model.add(Dense(256,activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(128,activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(64,activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(numclass,activation='softmax'))
    elif case==4:
        top_model.add(Dense(256,activation='relu'))
        top_model.add(Dropout(0.3))
        top_model.add(Dense(128,activation='relu'))
        top_model.add(Dropout(0.3))
        top_model.add(Dense(64,activation='relu'))
        top_model.add(Dropout(0.3))
        top_model.add(Dense(32,activation='relu'))
        top_model.add(Dropout(0.3))
        top_model.add(Dense(16,activation='relu'))
        top_model.add(Dropout(0.3))
        top_model.add(Dense(numclass,activation='softmax'))
    elif case==5:
        top_model.add(Dense(256,activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(128,activation='relu'))
        top_model.add(BatchNormalization())
        top_model.add(Dense(64,activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(numclass,activation='softmax'))
    elif case==6:
        top_model.add(Dense(256,activation='relu'))
        top_model.add(BatchNormalization())
        top_model.add(Dense(numclass,activation='softmax'))
    
    #ResNet50とオリジナル部分を結合してモデルの作成
    model=Model(inputs=resnet50.input,
                outputs=top_model(resnet50.output))
    
   # resnet50.summary()
    model.summary()
    

    #resnet50部分の重みを固定（転移学習）
    for layer in resnet50.layers:
        layer.trainable=False
    
    #多クラス分類を指定
    model.compile(loss='binary_crossentropy',
                    optimizer=optimizer,
                    metrics=['accuracy'])
    
    return model
    

import pickle
def save_model_data(save_name,model,hist,test_img):
    #モデル構造の保存
    open(save_name +"model.json","w").write(model.to_json())

    #学習済みの重みを保存
    model.save_weights(save_name+"weight.hdf5")

    #学習履歴を保存
    with open(save_name+"result.jdon","wb") as f:
        pickle.dump(hist.history,f)
        
    #  予測確率を保存
    with open(save_name + "preds.jdon","wb") as f:
        pickle.dump(model.predict(test_img),f)


In [ ]:
#評価の関数
#学習歴を保存
def plot_history(history,save_path,fig_size_width,fig_size_height,lim_font_size):
    #  まずは訓練
    acc=history.history['accuracy']
    loss =history.history['loss']
   
    epochs=range(len(acc))
    
    plt.figure(figsize=(fig_size_width,fig_size_height))
    plt.rcParams['font.family']='Times New Roman'
    plt.rcParams['font.size']=lim_font_size
    
    plt.plot(epochs,acc,color="blue",linestyle="solid",label='train acc') 
    plt.plot(epochs,loss,color="red",linestyle="dashed",label='train loss')
    plt.ylim(bottom=0)
   
    train_graph=save_path+'hist_train_record.png'
    
    plt.legend()
    plt.grid()
    
    plt.savefig(train_graph)
    plt.close()
    
#     次は検証
    plt.figure(figsize=(fig_size_width,fig_size_height))
    plt.rcParams['font.family']='Times New Roman'
    plt.rcParams['font.size']=lim_font_size
    
    val_acc=history.history['val_accuracy']
    val_loss=history.history['val_loss']
    
    plt.plot(epochs,val_acc,color="green",linestyle="dashdot",label='valid acc')
    plt.plot(epochs,val_loss,color="orange",linestyle="dotted",label='valid loss')
    plt.ylim(bottom=0)
    
    valid_graph=save_path+'hist_valid_record.png'
    
    plt.legend()
    plt.grid()
    
    plt.savefig(valid_graph)
    plt.close()
    
#     最後は混合
    plt.figure(figsize=(fig_size_width,fig_size_height))
    plt.rcParams['font.family']='Times New Roman'
    plt.rcParams['font.size']=lim_font_size

    plt.plot(epochs,acc,color="blue",linestyle="solid",label='train acc')
    plt.plot(epochs,val_acc,color="green",linestyle="dashdot",label='valid acc')
    plt.plot(epochs,loss,color="red",linestyle="dashed",label='train loss')
    plt.plot(epochs,val_loss,color="orange",linestyle="dotted",label='valid loss')
    plt.ylim(bottom=0)
    
    mix_graph = save_path+'hist.png'
    
    plt.legend()
    plt.grid()
    
    plt.savefig(mix_graph)
    plt.close()
    
#     accのみ
    plt.figure(figsize=(fig_size_width,fig_size_height))
    plt.rcParams['font.family']='Times New Roman'
    plt.rcParams['font.size']=lim_font_size

    plt.plot(epochs,acc,color="blue",linestyle="solid",label='train acc')
    plt.plot(epochs,val_acc,color="green",linestyle="dashdot",label='valid acc')
    plt.ylim(bottom=0)
    
    acc_graph = save_path+'acc.png'
    
    plt.legend()
    plt.grid()
    
    plt.savefig(acc_graph)
    plt.close()
    
#     lossのみ
    plt.figure(figsize=(fig_size_width,fig_size_height))
    plt.rcParams['font.family']='Times New Roman'
    plt.rcParams['font.size']=lim_font_size
    
    plt.plot(epochs,loss,color="red",linestyle="dashed",label='train loss')
    plt.plot(epochs,val_loss,color="orange",linestyle="dotted",label='valid loss')
    plt.ylim(bottom=0)
    
    loss_graph = save_path+'loss.png'
    
    plt.legend()
    plt.grid()
    
    plt.savefig(loss_graph)
    plt.close()


##コンフュージョンマトリックス
from sklearn.metrics import confusion_matrix 
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,roc_auc_score
import numpy as np

def Confusion_matrix(model,target,label,cm_name,color,view_name):
    #  予測の出力　　リストの０番目の要素の方が大きいときは陰性予想、１番目の方が大きいときは陽性予想
    preds=model.predict(target)
    
    #  ０と１番目の要素を比べ値が大きいほうの要素番号をリストとして保存
    #  ラベルと照らし合わせるためにpredsから０と１の予想リストを作成
    pred_list=[]
    for pred in preds:
        pred_list.append(np.argmax(pred))
    
    tn,fp,fn,tp=confusion_matrix(label,pred_list).ravel()
    acc=metrics.accuracy_score(label,pred_list)  #  accuracy
    ppv=metrics.precision_score(label,pred_list) #  precision(精度、適合率)
    tpr=metrics.recall_score(label,pred_list)    #　recall ,true positive rate(再現率、真陽性率)
    tnr=tn/(tn+fp)                               #  true negative rate(真陰性率、特異度)
    fnr=fn/(tp+fn)                               #  false negative rate(偽陰性率)
    fpr=fp/(fp+tn)                               #  false positive rate(偽陽性率)
    
    #  数値の書式を設定
    acc='{:.4f}'.format(acc)
    ppv='{:.4f}'.format(ppv)
    tpr='{:.4f}'.format(tpr)
    tnr='{:.4f}'.format(tnr)
    fnr='{:.4f}'.format(fnr)
    fpr='{:.4f}'.format(fpr)
    
    
    plt.figure()
    c_m=confusion_matrix(label,pred_list)  #  混合行列を取得
    #  混合行列のレイアウトを設定
    plt.text(0,3.0,view_name,size=17)
    plt.text(0.4,0.3,"TN")
    plt.text(1.4,0.3,"FP")
    plt.text(0.4,1.3,"FN")
    plt.text(1.4,1.3,"TP")
    plt.text(2.6,0.2,"ACC:"+str(acc))
    plt.text(2.6,0.6,"PPV:"+str(ppv))
    plt.text(2.6,1.0,"TPR:"+str(tpr))
    plt.text(2.6,1.4,"TNR:"+str(tnr))
    plt.text(2.6,1.8,"FNR:"+str(fnr))
    plt.text(2.6,2.2,"FPR:"+str(fpr)) 
    
    sns.heatmap(c_m,annot=True,cmap=color,fmt='g')
    plt.xlabel("predict")
    plt.ylabel("actual")
    plt.savefig(cm_name,bbox_inches='tight') #  描画範囲からはみ出ないようにする
    plt.close()
    return pred_list

# ROC曲線の作成
import numpy as np
from sklearn.metrics import confusion_matrix 
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import roc_curve,roc_auc_score

def ROC_CURVE(model,img,label,save_name,view_name):
#  閾値の列
    threshhold = np.arange(0,1,0.01)
    n = threshhold.size

#  データの作成
    a = model.predict(img)  #  0成分：陰性、1成分：陽性  predsが入る
    N = len(a)
#print(N)
    b = np.zeros((n,N))  #  閾値により０と１に振り分けた入れ物
    
#  陽性側が閾値より大きければ１とする
    for t in range(n):
        for i in range(N):
            if a[i][1]>threshhold[t]:
                b[t][i] = 1
            
#  auc値のために予測と正解ラベルの２重リストを一つのリストにする
    b_list=b.tolist()
#print(b)
            
    B=[] # 予測ラベル
    C=[] #  正解ラベル
    for t in range(n):
        for i in range(N):
            B.append(b_list[t][i])
            C.append(label[i])            
# print(C)
#print(len(C),len(B))
    
#  グラフにプロットする準備            
    fpr_np=[]
    tpr_np=[]
    for t in range(n):
        fpr_np.append(roc_curve(label,b[t])[0])
        tpr_np.append(roc_curve(label,b[t])[1])
        tn,fp,fn,tp=confusion_matrix(label,b[t]).ravel()
        print('Threshold=',f"{0+0.01*t:.2f}",'\tTPR=',f"{tp/(tp+fn):.2f}",'\tFPR=',f"{fp/(fp+tn):.2f}")
        fpr=np.array(fpr_np)
        tpr=np.array(tpr_np)
    print(fpr)
    print(tpr)

    tprfpr=np.zeros((n,3))
    for t in range(n):
        tprfpr[t][0]=0+0.01*t
        tprfpr[t][1]=tpr[t][1]
        tprfpr[t][2]=fpr[t][1]
    with open(save_name + "tprfpr.jdon","wb") as f:
        pickle.dump(tprfpr,f)

#  グラフにプロット
    plt.figure()
    plt.xlabel('FPR:False positive rate')
    plt.ylabel('TPR:True positive rate')
    plt.grid()
    roc=roc_auc_score(C,B)
    plt.text(0.5,0.0," auc :"+str('{:.4f}'.format(roc)))
    plt.title(view_name+"\n",fontsize=17)
    plt.xlim([0,1])
    plt.ylim([0,1])
    for i in range(n):
        plt.plot(fpr[i][1],tpr[i][1],color="b",marker='o',alpha=0.3)
    plt.savefig(save_name+'tkt_roc_curve.png',bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
# check path
import os
from sklearn.model_selection import KFold
for rate_cnt in range(1):
    if rate_cnt == 1:
        rate = 1e-3
        
    elif rate_cnt == 0:
        rate = 1e-4
        
    elif rate_cnt == 2:
        rate = 1e-5
        
    elif rate_cnt == 3:
        rate = 1e-6
        
    for batch_cnt in range(5):
        if batch_cnt == 0:
            batch = 16
            
        elif batch_cnt == 1:
            batch = 32
            
        elif batch_cnt == 2:
            batch = 64
            
        elif batch_cnt == 3:
            batch = 128
            
        elif batch_cnt == 4:
            batch = 256
        
        kf = KFold(n_splits=5)
        cnt=1
        for train_idx, val_idx in kf.split(trainvalid_0_img2):
            train_img = np.concatenate([trainvalid_0_img2[train_idx], trainvalid_1_img2[train_idx]])
            train_label_ndarray = np.concatenate([trainvalid_label_0_ndarray[train_idx], trainvalid_label_1_ndarray[train_idx]])
            valid_img = np.concatenate([trainvalid_0_img2[val_idx], trainvalid_1_img2[val_idx]])
            valid_label_ndarray = np.concatenate([trainvalid_label_0_ndarray[val_idx], trainvalid_label_1_ndarray[val_idx]])
            train_label = []
            valid_label = []
            for i in train_label_ndarray:
                train_label.append(i)
            for t in valid_label_ndarray:
                valid_label.append(t)
            train_amount=len(train_label)
            from keras import optimizers
            if opt=="Adam":
                optimizer=optimizers.Adam(lr=rate)
            elif opt=='AMSGrad':
                optimizer=optimizers.Adam(lr=rate,amsgrad=True)
            elif opt=="RMS":
                optimizer=optimizers.RMSprop(lr=rate)
            elif opt=='SGD':
                optimizer=optimizers.SGD(lr=rate)
            elif opt=='adamax':
                optimizer=optimizers.Adamax(lr=rate)
                
            save_name,save_view_name=folder_path(result_folder,date_folder,title,models,case,opt,dataset,train_amount,cnt,rate,batch,epochs)
            print("-----------------------------------------------")
            print(save_name)
            print("png_viwe:  "+save_view_name)
            print("- - - - - - - - - - - - - - - - - - - - - - - -")
            if (os.path.exists(save_name)==True):
                print("exist// skip")
            cnt+=1

In [ ]:
learn_start=time.time()
size=224
for rate_cnt in range(4):
    if rate_cnt == 0:
        rate = 1e-7
    
    elif rate_cnt == 1:
        rate = 1e-4
        
    elif rate_cnt == 2:
        rate = 1e-5
        
    elif rate_cnt == 3:
        rate = 1e-6
        
    for batch_cnt in range(4):
        if batch_cnt == 0:
            batch = 16
            
        elif batch_cnt == 1:
            batch = 32
            
        elif batch_cnt == 2:
            batch = 64
            
        elif batch_cnt == 3:
            batch = 128
            
        # elif batch_cnt == 4:
        #     batch = 256
        
        kf = KFold(n_splits=5)
        cnt=1
        for train_idx, val_idx in kf.split(trainvalid_0_img2):
            train_img = np.concatenate([trainvalid_0_img2[train_idx], trainvalid_1_img2[train_idx]])
            train_label_ndarray = np.concatenate([trainvalid_label_0_ndarray[train_idx], trainvalid_label_1_ndarray[train_idx]])
            valid_img = np.concatenate([trainvalid_0_img2[val_idx], trainvalid_1_img2[val_idx]])
            valid_label_ndarray = np.concatenate([trainvalid_label_0_ndarray[val_idx], trainvalid_label_1_ndarray[val_idx]])
            train_label = []
            valid_label = []
            for i in train_label_ndarray:
                train_label.append(i)
            for t in valid_label_ndarray:
                valid_label.append(t)
            print("cnt",str(cnt),valid_label)
            #ワンホット変換 #学習用（hist）
            train_label_list = np_utils.to_categorical(train_label,num_class)
            valid_label_list = np_utils.to_categorical(valid_label,num_class)
            print("cnt",str(cnt),valid_label_list)
            
            train_amount=2425
            from keras import optimizers
            if opt=="Adam":
                optimizer=optimizers.Adam(lr=rate)
            elif opt=='AMSGrad':
                optimizer=optimizers.Adam(lr=rate,amsgrad=True)
            elif opt=="RMS":
                optimizer=optimizers.RMSprop(lr=rate)
            elif opt=='SGD':
                optimizer=optimizers.SGD(lr=rate)
            elif opt=='adamax':
                optimizer=optimizers.Adamax(lr=rate)
                
            save_name,save_view_name=folder_path(result_folder,date_folder,title,models,case,opt,dataset,train_amount,cnt,rate,batch,epochs)
            print("-----------------------------------------------")
            print(save_name)
            print("png_viwe:  "+save_view_name)
            print("- - - - - - - - - - - - - - - - - - - - - - - -")
            
            os.makedirs(save_name,exist_ok=True)
            print("make folder & learn start ")
    
            model=make_model(size,num_class,optimizer,case,mode,read_name)
        
            # コールバック関連設定
            # モデル保存用のディレクトリ
            model_dir = os.path.join(save_name,'models')
            os.makedirs(model_dir,exist_ok = True)
            weights_dir = os.path.join(model_dir,'weights')
            os.makedirs(weights_dir,exist_ok = True)
           # 重みを1エポックごとに保存
            cp_filepath =  os.path.join(weights_dir, 'ep_{epoch:02d}_vls_{val_loss:.1f}.h5')
            cp = ModelCheckpoint(cp_filepath, 
                                 monitor='val_loss', 
                                 verbose=0,
                                 save_best_only=False, 
                                 save_weights_only=True, 
                                 mode='auto', 
                                 period=5
                                 )
            # EaelyStoppingの設定
            early_stopping = EarlyStopping(monitor='loss',
                                           min_delta=0.2,
                                           patience=30
                                           )
            
            # 学習の実行
            hist=model.fit(train_img,
                           train_label_list,
                           batch_size=batch,
                           epochs=epochs,
                           validation_data=(valid_img,valid_label_list),
                           callbacks=[cp,early_stopping]
                           )
            # いろいろ保存
            save_model_data(model_dir,model,hist,test_img)
            # 学習曲線
            fig_size_Width=12
            fig_size_Height=10
            fig_Font_size=25
            plot_history(hist,
                         save_path=save_name,
                         fig_size_width=fig_size_Width,
                         fig_size_height=fig_size_Height,
                         lim_font_size=fig_Font_size)
            cm_file_name=save_name+"CM_train.png"
            color="Reds"
            pred_list=Confusion_matrix(model,train_img,train_label,cm_file_name,color,save_view_name)
    
            cm_file_name=save_name+"CM_valid.png"
            color="Blues"
            pred_list=Confusion_matrix(model,valid_img,valid_label,cm_file_name,color,save_view_name)

            color='Greens'
            cm_file_name=save_name+"CM_test.png"
            pred_list=Confusion_matrix(model,test_img,test_label,cm_file_name,color,save_view_name)
    
            #  閾値変化のroc曲線
            ROC_CURVE(model,test_img,test_label,save_name,save_view_name)
            cnt+=1

In [ ]:
learn_time=time.time()-learn_start
learn_time/=60
print("{:.04}min in learn".format(learn_time))
print("finish")